In [1]:
import pymongo
from redis import Redis

REDIS = Redis(host='this_redis')
mongo_client = pymongo.MongoClient('this_mongo')
corpus_db = mongo_client.corpus

In [2]:
cd ..

/home/jovyan


In [3]:
import pandas as pd

In [4]:
corpus = pd.read_pickle('data/corpus.p')

In [5]:
corpus_df_records = corpus.to_dict('records')

In [6]:
corpus_df_records[:4]

[{'sentence': 'In the great green room There was a telephone And a red balloon',
  'title': 'Goodnight, Moon'},
 {'sentence': 'And a picture of-', 'title': 'Goodnight, Moon'},
 {'sentence': 'The cow jumping over the moon', 'title': 'Goodnight, Moon'},
 {'sentence': 'And there were three little bears sitting on chairs And two little kittens',
  'title': 'Goodnight, Moon'}]

In [7]:
corpus_db.documents.drop()
corpus_db.documents.insert_many(corpus_df_records)

In [8]:
corpus_db.documents.count()

170

In [9]:
corpus_db.documents.find_one({'tokens' : {'$exists': False}})

{'_id': ObjectId('5a91c5da04506b00c082add0'),
 'sentence': 'In the great green room There was a telephone And a red balloon',
 'title': 'Goodnight, Moon'}

# `MAPPER`

In [10]:
def tokenize(document):
    return (document
            .replace(',','')
            .replace('.','')
            .split())

def MAPPER(document):
    for word in tokenize(document):
        yield (word, 1)

In [11]:
doc = corpus_db.documents.find_one({'tokens' : {'$exists': False}})
list(MAPPER(doc['sentence']))

[('In', 1),
 ('the', 1),
 ('great', 1),
 ('green', 1),
 ('room', 1),
 ('There', 1),
 ('was', 1),
 ('a', 1),
 ('telephone', 1),
 ('And', 1),
 ('a', 1),
 ('red', 1),
 ('balloon', 1)]

In [12]:
unprocessed_filter = {'processed' : {'$exists': False}}
doc = corpus_db.documents.find_one(unprocessed_filter)
while doc:
    id_filter = { '_id' : doc['_id'] }
    tokens = list(MAPPER(doc['sentence']))
    update = { '$set' : {'tokens' : tokens, 'processed' : 'tokenized'} }
    corpus_db.documents.update_one(id_filter, update)
    doc = corpus_db.documents.find_one({'tokens' : {'$exists': False}})

In [13]:
corpus_db.documents.find_one()

{'_id': ObjectId('5a91c5da04506b00c082add0'),
 'processed': 'tokenized',
 'sentence': 'In the great green room There was a telephone And a red balloon',
 'title': 'Goodnight, Moon',
 'tokens': [['In', 1],
  ['the', 1],
  ['great', 1],
  ['green', 1],
  ['room', 1],
  ['There', 1],
  ['was', 1],
  ['a', 1],
  ['telephone', 1],
  ['And', 1],
  ['a', 1],
  ['red', 1],
  ['balloon', 1]]}

# `COLLECTOR`

In [23]:
def COLLECTOR(document, vocabulary):
    for token in doc['tokens']:
        REDIS.sadd(vocabulary, token[0])
        REDIS.rpush(token[0], token[1])

In [24]:
tokenized_filter = {'processed' : 'tokenized'}
doc = corpus_db.documents.find_one(tokenized_filter)
doc

In [25]:
while doc:
    id_filter = { '_id' : doc['_id'] }
    tokens = doc['tokens']
    update = { '$set' : {'processed' : 'counted'} }
    COLLECTOR(doc, 'corpus_vocab')
    corpus_db.documents.update_one(id_filter, update)
    doc = corpus_db.documents.find_one(tokenized_filter)

In [26]:
vocabulary = REDIS.smembers('corpus_vocab')
list(vocabulary)[:5]

[b'back', b'sight', b'"I\'ve', b'"This', b'money?"']

In [27]:
for word in list(vocabulary)[:5]:
    print(REDIS.lrange(word, 0, -1))

[b'1', b'1', b'1', b'1', b'1', b'1', b'1', b'1']
[b'1', b'1']
[b'1', b'1', b'1', b'1', b'1', b'1']
[b'1', b'1', b'1', b'1', b'1', b'1']
[b'1', b'1']


# `REDUCER`

In [19]:
def REDUCER(word):
    counts = [int(i) for i in REDIS.lrange(word, 0, -1)]
    return sum(counts)

In [20]:
word_counts = []
for word in vocabulary:
    word_counts.append((word.decode(), REDUCER(word)))

In [21]:
word_counts.sort(key=lambda x: x[1], reverse=True)

In [22]:
word_counts[:10]

[('and', 196),
 ('the', 194),
 ('a', 112),
 ('And', 82),
 ('said', 62),
 ('to', 60),
 ('he', 60),
 ('was', 48),
 ('tree', 42),
 ('boy', 40)]